In [53]:
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [54]:
ABSOLUTE_PATH = "../../datasets/"

In [55]:
df = pd.read_csv(ABSOLUTE_PATH+"flipkart-data.csv")

In [56]:
def convert_to_arr(row):
    s = row["product_category_tree"].strip('[]').strip('""')
    s = s.split(">>")
    
    for word in s:
        word = word.strip()
    return s

In [57]:
df["category_array"] = df.apply(convert_to_arr,axis=1)

In [58]:
def get_keywords(row):
    sentence = str(row["description"])
    arr = process_keywords(sentence)
    cat_arr = row["category_array"]
    for word in cat_arr:
        arr = arr + process_keywords(word)
    return arr

In [59]:
def process_keywords(sentence):
    sentence = sentence.lower()
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    words = tokenizer.tokenize(sentence)
    stop_words = set(stopwords.words('english')) 
    keywords = [w for w in words if not w in stop_words]
    return keywords
    

In [67]:
df["keywords_array"] = df.apply(get_keywords,axis=1)


KeyError: ('description', 'occurred at index 0')

In [68]:
df.drop("product_category_tree",inplace=True,axis=1)
df.drop("description",inplace=True,axis=1)

KeyError: "['product_category_tree'] not found in axis"

In [69]:
df.to_csv(ABSOLUTE_PATH+"category_dataset.csv")